In [1]:
%cd ..

C:\Users\kubak\Desktop\PhiloModel\github\AgoraRAG


C:\Users\kubak\AppData\Local\Programs\Python\Python312\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import os
import openai
import sys
from dotenv import load_dotenv, find_dotenv

In [3]:
sys.path.append('../..')

_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

# Load data

In [4]:
from my_code.load_data.import_pdfs import load_pdfs_from_dir

In [5]:
dir_path = f"../../data/pdfs/books/HistoriaFilozofii-Tatarkiewicz/"

In [6]:
all_pdfs = load_pdfs_from_dir(dir_path)

Processing ../../data/pdfs/books/HistoriaFilozofii-Tatarkiewicz\wadysaw-tatarkiewicz-historia-filozofii-tom1.pdf
Processing ../../data/pdfs/books/HistoriaFilozofii-Tatarkiewicz\wadysaw-tatarkiewicz-historia-filozofii-tom2.pdf
Loaded 443 docs.


# Splitting data

In [7]:
#!pip install chromadb

In [8]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    #separators=["\n\n", "\n", " ", ""], 
    chunk_size = 1000,
    chunk_overlap = 150
)

In [9]:
splits = text_splitter.split_documents(all_pdfs)

In [10]:
len(splits)

2362

In [11]:
splits[100].page_content

'Wszystko płynie, nic nie trwa, "niepodobna wstąpić dwukrotnie do tej samej rzeki", bo już inne  \nnapłynęły w nią wody. Obrazem rzeczywistości jest również śmierć: "obawiamy się jednej śmierci,  \na już wielu śmierciom ulegliśmy"; "dla duszy jest śmiercią stać się wodą, dla wody jest śmiercią  \nstać się ziemią". Natura jest ciągłą śmiercią i ciągłym rodzeniem się; ta sama natura jest ciągle  \ninna: "do tej samej rzeki wstępujemy i nie wstępujemy". Nie możemy powiedzieć, że jesteśmy, bo  \n"jesteśmy i nie jesteśmy zarazem";  prawdą jest tylko,  że się zmieniamy.  Wprawdzie niekiedy  \nrzeczy zdają się trwać, ale trwanie ich jest złudzeniem. Nie ma rzeczy o stałych własnościach; nie'

# Embedding

In [12]:
from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings()

C:\Users\kubak\AppData\Local\Temp\ipykernel_4792\1742550774.py:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings()


# Vectorstores

In [13]:
from langchain.vectorstores import Chroma

In [14]:
persist_directory = 'docs/chroma/'

In [15]:
! rm -rf ./docs/chroma  # remove old database files if any

In [16]:
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
)

In [17]:
print(f'Rozmiar vectorDB: {vectordb._collection.count()}')

2362


# Similiarity Search

In [18]:
question = "Kim był Kant?"

In [19]:
docs = vectordb.similarity_search(question, k=3)

In [20]:
docs[0].page_content[:100]

'specjalne  towarzystwo  kaniowskie  (Kantgesellschaft),  będące  głównym  zrzesze-niem \nfilozoficzny'

In [21]:
docs_mmr = vectordb.max_marginal_relevance_search(question, k=3)

In [22]:
docs_mmr[0].page_content[:100]

'specjalne  towarzystwo  kaniowskie  (Kantgesellschaft),  będące  głównym  zrzesze-niem \nfilozoficzny'

# Retriver

In [23]:
from langchain.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

In [24]:
metadata_field_info = [
    AttributeInfo(
        name="source",
        description="Źródło wiadomosci",
        type="string",
    ),
    AttributeInfo(
        name="page",
        description="Strona",
        type="integer",
    ),
]

In [25]:
document_content_description = "Streszczenie poglądów filozofa."

llm = OpenAI(model='gpt-3.5-turbo-instruct', temperature=0)
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectordb,
    document_content_description,
    metadata_field_info,
    verbose=True
)

C:\Users\kubak\AppData\Local\Temp\ipykernel_4792\3576603665.py:3: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(model='gpt-3.5-turbo-instruct', temperature=0)


In [26]:
question = "Kim był Kant?"

In [27]:
docs = retriever.get_relevant_documents(question)

C:\Users\kubak\AppData\Local\Temp\ipykernel_4792\10663643.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(question)


In [29]:
for i, doc in enumerate(docs):
    print(doc.page_content)

metafizyki moralności (Grundlegung zur Metaphysik der Sitten), 1785. Poza trzema krytykami i 
ich wariantami Kant ogłosił w tym okresie swą  filozofię dziejów (Idee zu einer allgemeinen 
Geschichte  in  weltburgerlicher  Absicht,  1784),  swą  filo-zofię  przyrody  (Metaphysische 
Anfangsgriinde der Naturwissenschaft, 1786) i swą filozofię religii (Die Religion innerhalb der 
Grenzen der blossen Vernunft, 1793). Po opuszczeniu katedry ogłosił niektóre ze swych wykładów 
uniwersyteckich: antropologię ogłosił sam w 1798, logikę opublikował zań Jasche; filozofię religii 
i metafizykę ogłosił Poltitz już po śmierci Kanta w 1817 i 1821, etykę zaś dopiero w 1924 r. 
Menzer.  Po  śmierci  Kanta  wy-dano  różne  jego  rękopisy,  przeważnie  notaty,  z  których 
najważniejsze są: Reflexionen, wyd. Erdmanna, 1882-1884, i Lose Blatter, wyd. Reickego, 1889-
1899.
ORIENTACJA.  Kant rozpoczął swą  twórczość  naukową  od  prac  przyrodniczych;  jako docent
nie uznał tego ulepszenia, nazywał je „hiper

# RetrievalQA chain

In [30]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate


In [63]:
# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "~ PhiloBot!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template,)

In [64]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

C:\Users\kubak\AppData\Local\Temp\ipykernel_4792\1956280058.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [65]:
from langchain.chains import ConversationalRetrievalChain
retriever=vectordb.as_retriever()

qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory
)

In [66]:
question = "Jak kant bronił istnienia sądów syntetycznych apriori?"
result = qa({"question": question})

In [70]:
result['answer']

'Kant bronił istnienia sądów syntetycznych a priori, opierając się na dwóch naukach: matematyce i czystym przyrodoznawstwie. Uważał, że sądy matematyczne są a priori i syntetyczne, niezależne od doświadczenia, ale przynoszące dodatkową wiedzę. Kant argumentował, że doświadczenie nie zapewnia pełnej powszechności i pewności, a tylko sądy a priori (takie jak syntetyczne) spełniają te kryteria. Jego zdaniem, ujęcie sądów syntetycznych a priori było istotne dla filozofii, chociaż zdanie to było przedmiotem kontrowersji wśród innych filozofów, takich jak empiryci.'

# GUI

In [80]:
import panel as pn
import param

class cbfs(param.Parameterized):
    chat_history = param.List([])
    answer = param.String("")
    db_query  = param.String("")
    db_response = param.List([])
    
    def __init__(self, qa, **params):
        super(cbfs, self).__init__( **params)
        self.panels = []
        self.loaded_file = "docs/cs229_lectures/MachineLearning-Lecture01.pdf"
        self.qa = qa
    
    def call_load_db(self, count):
        if count == 0 or file_input.value is None:  # init or no file specified :
            return pn.pane.Markdown(f"Loaded File: {self.loaded_file}")
        else:
            file_input.save("temp.pdf")  # local copy
            self.loaded_file = file_input.filename
            button_load.button_style="outline"
            self.qa = qa
            button_load.button_style="solid"
        self.clr_history()
        return pn.pane.Markdown(f"Loaded File: {self.loaded_file}")

    def convchain(self, query):
        if not query:
            return pn.WidgetBox(pn.Row('User:', pn.pane.Markdown("", width=600)), scroll=True)
        result = self.qa({"question": query, "chat_history": self.chat_history})
        self.chat_history.extend([(query, result["answer"])])
        self.db_query = result["generated_question"]
        self.db_response = result["source_documents"]
        self.answer = result['answer'] 
        self.panels.extend([
            pn.Row('User:', pn.pane.Markdown(query, width=600)),
            pn.Row('ChatBot:', pn.pane.Markdown(self.answer, width=600, style={'background-color': '#F6F6F6'}))
        ])
        inp.value = ''  #clears loading indicator when cleared
        return pn.WidgetBox(*self.panels,scroll=True)

    @param.depends('db_query ', )
    def get_lquest(self):
        if not self.db_query :
            return pn.Column(
                pn.Row(pn.pane.Markdown(f"Last question to DB:", styles={'background-color': '#F6F6F6'})),
                pn.Row(pn.pane.Str("no DB accesses so far"))
            )
        return pn.Column(
            pn.Row(pn.pane.Markdown(f"DB query:", styles={'background-color': '#F6F6F6'})),
            pn.pane.Str(self.db_query )
        )

    @param.depends('db_response', )
    def get_sources(self):
        if not self.db_response:
            return 
        rlist=[pn.Row(pn.pane.Markdown(f"Result of DB lookup:", styles={'background-color': '#F6F6F6'}))]
        for doc in self.db_response:
            rlist.append(pn.Row(pn.pane.Str(doc)))
        return pn.WidgetBox(*rlist, width=600, scroll=True)

    @param.depends('convchain', 'clr_history') 
    def get_chats(self):
        if not self.chat_history:
            return pn.WidgetBox(pn.Row(pn.pane.Str("No History Yet")), width=600, scroll=True)
        rlist=[pn.Row(pn.pane.Markdown(f"Current Chat History variable", styles={'background-color': '#F6F6F6'}))]
        for exchange in self.chat_history:
            rlist.append(pn.Row(pn.pane.Str(exchange)))
        return pn.WidgetBox(*rlist, width=600, scroll=True)

    def clr_history(self,count=0):
        self.chat_history = []
        return 


In [81]:
cb = cbfs(qa)

file_input = pn.widgets.FileInput(accept='.pdf')
button_load = pn.widgets.Button(name="Load DB", button_type='primary')
button_clearhistory = pn.widgets.Button(name="Clear History", button_type='warning')
button_clearhistory.on_click(cb.clr_history)
inp = pn.widgets.TextInput( placeholder='Enter text here…')

bound_button_load = pn.bind(cb.call_load_db, button_load.param.clicks)
conversation = pn.bind(cb.convchain, inp) 

jpg_pane = pn.pane.Image( './img/convchain.jpg')

tab1 = pn.Column(
    pn.Row(inp),
    pn.layout.Divider(),
    pn.panel(conversation,  loading_indicator=True, height=300),
    pn.layout.Divider(),
)
tab2= pn.Column(
    pn.panel(cb.get_lquest),
    pn.layout.Divider(),
    pn.panel(cb.get_sources ),
)
tab3= pn.Column(
    pn.panel(cb.get_chats),
    pn.layout.Divider(),
)
tab4=pn.Column(
    pn.Row( file_input, button_load, bound_button_load),
    pn.Row( button_clearhistory, pn.pane.Markdown("Clears chat history. Can use to start a new topic" )),
    pn.layout.Divider(),
    pn.Row(jpg_pane.clone(width=400))
)
dashboard = pn.Column(
    pn.Row(pn.pane.Markdown('# ChatWithYourData_Bot')),
    pn.Tabs(('Conversation', tab1), ('Database', tab2), ('Chat History', tab3),('Configure', tab4))
)
dashboard

Column
    [0] Row
        [0] Markdown(str)
    [1] Tabs
        [0] Column
            [0] Row
                [0] TextInput(placeholder='Enter text here…')
            [1] Divider()
            [2] ParamFunction(function, _pane=WidgetBox, defer_load=False, height=300, loading_indicator=True)
            [3] Divider()
        [1] Column
            [0] ParamMethod(method, _pane=Column, defer_load=False)
            [1] Divider()
            [2] ParamMethod(method, _pane=Str, defer_load=False)
        [2] Column
            [0] ParamMethod(method, _pane=WidgetBox, defer_load=False)
            [1] Divider()
        [3] Column
            [0] Row
                [0] FileInput(accept='.pdf')
                [1] Button(button_type='primary', name='Load DB')
                [2] ParamFunction(function, _pane=Markdown, defer_load=False)
            [1] Row
                [0] Button(button_type='warning', name='Clear History')
                [1] Markdown(str)
            [2] Divider()
    